In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import missingno as msnco

In [ ]:
#진행상황
!pip install tqdm
import time
from tqdm import tqdm

# for i in tqdm(range(10)):
# #     time.sleep(0.1)
# empty_list = []
# for word in tqdm(['apple', 'ball', 'cat', 'doll', 'egg'], desc='iterate list'):
#     time.sleep(0.1)
#     empty_list.append(word)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#과정  
1. 데이터 탐색(EDA)
 - 데이터 분포 조사(히스토그램, 데이터 수, 이상치 조사, 결측치)
 - 데이터 타입 확인
 - 데이터 상관 관계(joint plot, heatmap)
 - 데이터 시각화

2. 데이터 전처리(수치형 데이터 기준)
 - 결측치 처리(drop, imputation(결측값 대체))
 - 이상치 처리(정규분포, box plot, ML)
 - 스케일링

3. 데이터 분할(data split) - train,test 데이터가 나뉘어져있으므로 할 필요x

4. 모델 구축
 - 모델 선택
 - 모델 하이퍼 파라미터 설정

5. 학습
 - cost(mse, cross entropy)
 - 학습 방법(grid search, 경사하강법)

6. 검증
 - 평가지표
 - 하이퍼 파라미터 튜닝
 - cross validation 

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/dacon/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon/test.csv')

In [3]:
sample_df = pd.read_csv('/content/drive/MyDrive/dacon/sample_submission.csv')

In [4]:
train_df.head(2)

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN


In [ ]:
test_df.head(2)

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,SAT,14683,Texas,191.0,United Air Lines Inc.,UA,NaN,N79402
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,ATL,10397,NaN,746.0,Delta Air Lines Inc.,DL,19790.0,N3765


In [ ]:
sample_df.head() 

,ID,Not_Delayed,Delayed
0,TEST_000000,0,1
1,TEST_000001,0,1
2,TEST_000002,0,1
3,TEST_000003,0,1
4,TEST_000004,0,1


In [ ]:
#train_df 먼저 

In [ ]:
train_df.head(3)

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN


In [ ]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [5]:
train_df[train_df['Estimated_Arrival_Time'].isna() & train_df['Estimated_Departure_Time'].isna()] #출발예정시간과 도착예정시간 둘다 존재하지 않는 데이터가 11688개.
#출발예정시간 109019개와 도착예정시간109040개가 데이터가 존재하지 않는데 10만개의 데이터중 약 만개 정도만 출발,도착 예정시간 둘다 존재 하지 않음

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
109,TRAIN_000109,3,29,NaN,NaN,0,0,PDX,14057,Oregon,DEN,11292,Colorado,991.0,Southwest Airlines Co.,WN,19393.0,N499WN,NaN
277,TRAIN_000277,8,9,NaN,NaN,0,0,MKE,13342,Wisconsin,IAH,12266,Texas,984.0,Mesa Airlines Inc.,UA,20378.0,N88335,NaN
435,TRAIN_000435,9,12,NaN,NaN,0,0,OAK,13796,California,ORD,13930,Illinois,1836.0,Spirit Air Lines,NK,20416.0,N618NK,NaN
437,TRAIN_000437,7,12,NaN,NaN,0,0,HOU,12191,Texas,MSY,13495,NaN,302.0,Southwest Airlines Co.,WN,19393.0,N265WN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999771,TRAIN_999771,7,10,NaN,NaN,0,0,FLL,11697,Florida,ORD,13930,Illinois,1182.0,JetBlue Airways,B6,NaN,N529JB,NaN
999777,TRAIN_999777,9,27,NaN,NaN,0,0,STS,15023,California,SEA,14747,Washington,618.0,SkyWest Airlines Inc.,AS,20304.0,N403SY,Delayed
999815,TRAIN_999815,12,1,NaN,NaN,0,0,SEA,14747,Washington,BOI,10713,Idaho,399.0,NaN,AS,19687.0,N401QX,NaN
999911,TRAIN_999911,7,14,NaN,NaN,0,0,HOU,12191,Texas,DEN,11292,Colorado,883.0,Southwest Airlines Co.,WN,19393.0,N935WN,NaN


In [6]:
train_df.nunique() 
#Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 
#Origin_Airport와 Origin_Airpirt_ID는 동일한 정보를 나타내므로 둘 중 하나를 drop
#Destination_Airport와 Destination_Airport_ID는 동일한 정보를 나타내므로 둘 중 하나 drop

ID                          1000000
Month                            12
Day_of_Month                     31
Estimated_Departure_Time       1365
Estimated_Arrival_Time         1428
Cancelled                         1
Diverted                          1
Origin_Airport                  374
Origin_Airport_ID               374
Origin_State                     52
Destination_Airport             375
Destination_Airport_ID          375
Destination_State                52
Distance                       1597
Airline                          28
Carrier_Code(IATA)               11
Carrier_ID(DOT)                  28
Tail_Number                    6430
Delay                             2
dtype: int64

In [7]:
train_df.drop(['Cancelled','Diverted'], axis=1, inplace= True) #Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 

In [8]:
#Origin_Airport와 Origin_Airpirt_ID는 동일한 정보를 나타내므로 둘 중 하나를 drop
train_df.drop('Origin_Airport', axis=1, inplace= True)

In [9]:
#Destination_Airport와 Destination_Airport_ID는 동일한 정보를 나타내므로 둘 중 하나 drop
train_df.drop('Destination_Airport', axis =1, inplace= True)

In [10]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [11]:
train_df.nunique()

ID                          1000000
Month                            12
Day_of_Month                     31
Estimated_Departure_Time       1365
Estimated_Arrival_Time         1428
Origin_Airport_ID               374
Origin_State                     52
Destination_Airport_ID          375
Destination_State                52
Distance                       1597
Airline                          28
Carrier_Code(IATA)               11
Carrier_ID(DOT)                  28
Tail_Number                    6430
Delay                             2
dtype: int64

In [12]:
train_df[train_df['Origin_State'].isna() & train_df['Destination_State'].isna()] #출발주와 도착주 둘다 데이터가 존재하지 않는 개수 11996개

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
18,TRAIN_000018,8,11,1215.0,1258.0,11146,NaN,13930,NaN,416.0,NaN,NaN,20046.0,N440AW,NaN
101,TRAIN_000101,2,23,1515.0,1700.0,14783,NaN,13930,NaN,438.0,Air Wisconsin Airlines Corp,NaN,20046.0,N440AW,NaN
203,TRAIN_000203,2,23,940.0,1806.0,14771,NaN,11618,NaN,2565.0,Virgin America,VX,21171.0,N633VA,NaN
206,TRAIN_000206,2,26,NaN,2344.0,12264,NaN,10529,NaN,326.0,Mesa Airlines Inc.,UA,20378.0,N522LR,NaN
594,TRAIN_000594,2,26,920.0,950.0,11066,NaN,15016,NaN,409.0,Southwest Airlines Co.,WN,19393.0,N916WN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999215,TRAIN_999215,2,19,2241.0,2348.0,13487,NaN,11337,NaN,144.0,SkyWest Airlines Inc.,DL,20304.0,N603SK,NaN
999238,TRAIN_999238,10,26,706.0,904.0,10693,NaN,13930,NaN,409.0,NaN,AA,19805.0,N908AN,NaN
999698,TRAIN_999698,10,6,1932.0,NaN,10821,NaN,10721,NaN,369.0,Spirit Air Lines,NK,20416.0,N614NK,NaN
999763,TRAIN_999763,8,14,1009.0,1143.0,11278,NaN,13296,NaN,406.0,Republic Airlines,NaN,20452.0,N428YX,NaN


In [13]:
train_df[train_df['Airline'].isna() & train_df['Carrier_Code(IATA)'].isna() & train_df['Carrier_ID(DOT)']] #

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
18,TRAIN_000018,8,11,1215.0,1258.0,11146,NaN,13930,NaN,416.0,NaN,NaN,20046.0,N440AW,NaN
112,TRAIN_000112,2,21,2020.0,2347.0,13487,Minnesota,11278,Virginia,931.0,NaN,NaN,19790.0,N324US,Not_Delayed
243,TRAIN_000243,4,21,1350.0,1455.0,12954,California,12889,Nevada,231.0,NaN,NaN,19393.0,N462WN,NaN
266,TRAIN_000266,2,5,1155.0,1330.0,10721,Massachusetts,11618,New Jersey,200.0,NaN,NaN,19977.0,N69835,Not_Delayed
373,TRAIN_000373,4,29,NaN,1630.0,14107,Arizona,13198,Missouri,1044.0,NaN,NaN,19393.0,N7719A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999400,TRAIN_999400,3,18,945.0,1215.0,10693,Tennessee,11042,Ohio,448.0,NaN,NaN,19393.0,N7812G,NaN
999585,TRAIN_999585,4,30,816.0,934.0,10397,Georgia,11641,North Carolina,331.0,NaN,NaN,20363.0,N8694A,NaN
999688,TRAIN_999688,3,20,715.0,815.0,10800,California,12889,Nevada,223.0,NaN,NaN,19393.0,N423WN,NaN
999791,TRAIN_999791,9,8,1305.0,1605.0,11292,Colorado,11259,Texas,651.0,NaN,NaN,19393.0,N218WN,Not_Delayed


In [14]:
train_df[train_df['Carrier_Code(IATA)']== 'UA']

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
1,TRAIN_000001,8,15,740.0,1024.0,13930,Illinois,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
3,TRAIN_000003,7,10,905.0,1735.0,12892,California,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,14771,California,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
5,TRAIN_000005,4,13,1545.0,NaN,11618,NaN,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed
6,TRAIN_000006,1,20,1742.0,1903.0,11618,New Jersey,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999972,TRAIN_999972,5,26,1130.0,1333.0,10599,Alabama,12266,Texas,562.0,ExpressJet Airlines Inc.,UA,20366.0,N11547,NaN
999984,TRAIN_999984,8,4,834.0,1010.0,14869,Utah,11292,Colorado,391.0,SkyWest Airlines Inc.,UA,20304.0,N203SY,NaN
999989,TRAIN_999989,1,12,1717.0,2019.0,11618,New Jersey,14027,Florida,1023.0,United Air Lines Inc.,UA,19977.0,N18223,NaN
999992,TRAIN_999992,12,30,1307.0,1707.0,12441,Wyoming,13930,Illinois,1162.0,United Air Lines Inc.,UA,19977.0,N17719,Not_Delayed


In [15]:
train_df[train_df['Airline']=='United Air Lines Inc.'].head()	 # airline과 carrier_id는 항공사와 항공사 id를 표현한 것으로 결측치를 항공사와 항공사에 맞는 id를 넣은 후 -- 둘중 하나  drop 

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
3,TRAIN_000003,7,10,905.0,1735.0,12892,California,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
6,TRAIN_000006,1,20,1742.0,1903.0,11618,New Jersey,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed
31,TRAIN_000031,8,19,600.0,847.0,12889,NaN,11292,Colorado,628.0,United Air Lines Inc.,UA,19977.0,N17245,NaN
43,TRAIN_000043,9,30,1325.0,1506.0,14679,California,14771,California,447.0,United Air Lines Inc.,UA,NaN,N471UA,Not_Delayed
54,TRAIN_000054,3,12,1800.0,2109.0,11618,New Jersey,12892,California,2454.0,United Air Lines Inc.,UA,NaN,N14107,NaN


In [16]:
train_df.isna().sum() # 항공사, 항공사 id, 항공사 코드는 서로 연관이 있기 때문에 결측치를 채워 넣을 수 있음 ///
#출발예정시간, 도착 예정시간 -- 이거를  
#도착주 , 출발주  -- 

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [16]:
train_df.head(3)

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,13851,Oklahoma,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,13930,Illinois,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,11057,North Carolina,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN


In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Origin_State              890985 non-null   object 
 7   Destination_Airport_ID    1000000 non-null  int64  
 8   Destination_State         890921 non-null   object 
 9   Distance                  1000000 non-null  float64
 10  Airline                   891080 non-null   object 
 11  Carrier_Code(IATA)        891010 non-null   object 
 12  Carrier_ID(DOT)           891003 non-null   float64
 13  Tail_Number               10

In [18]:
train_df[train_df['Delay']=='Delayed']

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
10,TRAIN_000010,8,13,1730.0,1844.0,11278,Virginia,14122,Pennsylvania,204.0,Republic Airlines,AA,NaN,N119HQ,Delayed
55,TRAIN_000055,8,18,1700.0,1900.0,12191,Texas,10693,Tennessee,670.0,Southwest Airlines Co.,NaN,19393.0,N8658A,Delayed
95,TRAIN_000095,3,14,1905.0,2159.0,14027,NaN,12953,New York,1035.0,JetBlue Airways,B6,20409.0,N337JB,Delayed
118,TRAIN_000118,4,11,1935.0,2115.0,10397,Georgia,13487,Minnesota,907.0,Delta Air Lines Inc.,DL,19790.0,N912DU,Delayed
162,TRAIN_000162,12,16,2040.0,2249.0,14747,Washington,14831,California,696.0,Alaska Airlines Inc.,AS,19930.0,N551AS,Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999816,TRAIN_999816,11,6,2115.0,2348.0,13930,Illinois,11076,Michigan,360.0,SkyWest Airlines Inc.,UA,20304.0,N964SW,Delayed
999843,TRAIN_999843,6,13,1749.0,1850.0,13342,Wisconsin,13930,Illinois,67.0,Trans States Airlines,UA,20237.0,N12136,Delayed
999955,TRAIN_999955,4,12,1310.0,1356.0,14747,Washington,14057,Oregon,129.0,Horizon Air,AS,19687.0,N432QX,Delayed
999963,TRAIN_999963,5,2,1759.0,1926.0,12953,New York,11278,Virginia,214.0,NaN,DL,20452.0,N871RW,Delayed


In [ ]:
#딜레이 된 데이터와 딜레이 되지 않은 데이터 , 딜레이 여부가 존재하지 않는 데이터 이렇게 3종류가 있다고 할 때 
# 각각 돌려서 

그냥 생각해본거 --  
항공사 별로 -- 이게 비행기 id같은게 있으면 (이게 tail_number인가) 

1. 이게 딜레이 여부가 거리와 관련이 있다고 생각되는데 항공사(비행기)별로 같은 거리라 하더라도 어떤 항공사 비행기는 빨리 가고 어떤 항공사 비행기는 느리게 갈 수 있지 않을까 -- 싶어서 
2. 거리와 항공사(비행기)이게 관련이 있다면 // 우선 출발예정시간과 도착예정시간으로 비행시간을 알수 있고 --> train_df에 비행시간 추가하기 : 도착예정시간- 출발예정시간
3. 데이터 프레임에 비행시간을 추가한 뒤 -- 20만개 정도가 출발 혹은 도착예정 시간 데이터가 존재하지 않음. > 거리와 비행시간이 항공사와 관계없이 비례한다면 --> 거리 데이터는 결측치가 존재하지 않으니 --> 모든 데이터에 비행시간을 추가해줄 수 있음 --> 그렇다면 출발예정시간이나 도착예정시간 둘 중 한 데이터만 존재한다면 비행시간을 알기 때문에 다른 시간을 알수 있으므로 --> 출발, 도착예정시간 둘다 없는 데이터를 제외한 대부분의 데이터의 결측치를 보완할수 있음
4. 문제는 거리와 비행시간이 항공사에 따라 달라질 때 -- -이럴때는 어떻게 해야할지--- 이건 회의때 말해보고 
5. 그냥 거리에 따라 비행시간이 비례한다고 가정하면 만개 정도의 데이터를 제외하고는 대부분의 결측치가 채워짐 -- 

6. 출발주, 도착주  --- > 출발 공항 id, 도착 공항 id와 관련이 있긴 한데 -- 만약 texas라 가정하면 위에 < train_df[train_df['Delay']=='Delayed'] > 에서만 해도 25개나 존재 -- 이걸 그냥 삭제 해버리기에는 출발공항과 도착 공항이 필요할거 같은데 -- 

#####     
1. 딜레이 여부와 거리가 관련이 있다고 가정-- 
2. 거리와 항공사(비행기)가 관련이 있는지 없는지 -- 관련이 있다면 같은 거리를 비행한다고 하더라도 항공사 별로 비행시간의 차이가 있지 않을까 
3. 거리와 항공사가 관련이 없다면 -- 거리와 비행시간은 비례한다고 가정
4. 출발예정시간과 도착예정시간으로 비행예정시간을 알아낼 수 있고 이를 데이터프레임에 비행예정시간 컬럼을 새로 추가 
5. 거리와 비행시간이 비례한다고 가정했으므로 거리정보는 모든 데이터가 존재하므로 모든 데이터에 비행시간을 추가할수 있음 -- 문제점) 그럼 거리와 비행시간이 비례하면 비행시간을 넣어야 하는 이유가 있을까  -- 
6. 비행시간을 알기 때문에 출발시간을 알면 도착시간을 알수 있고, 도착시간을 알면 출발시간을 알수 있음  -- 출발시간과 도착시간 둘다 모르는 약 만개의 데이터만 제외하고 대부분의 결측치를 채울 수 있음   -- ) 비행시간으로 출발,도착시간을 채워넣기 위함? 

7. 출발주, 도착주  --> 출발공항id와 도착공항id와 관련은 있으나 그냥 drop하기에는 필요한 데이터지 않을까 싶은데 -- > 출발주가 texas라 가정하면 그에 맞는 출발공항id가 25개나 존재 --  
 


In [21]:
# import os

In [22]:
# os.chdir('/content/drive/MyDrive/dacon')

In [23]:
# train_df.to_csv('train_df.csv')

In [20]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,13851,Oklahoma,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,13930,Illinois,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,11057,North Carolina,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,12892,California,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,14771,California,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,13930,NaN,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,11637,NaN,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,13796,NaN,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,10693,Tennessee,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Origin_State              890985 non-null   object 
 7   Destination_Airport_ID    1000000 non-null  int64  
 8   Destination_State         890921 non-null   object 
 9   Distance                  1000000 non-null  float64
 10  Airline                   891080 non-null   object 
 11  Carrier_Code(IATA)        891010 non-null   object 
 12  Carrier_ID(DOT)           891003 non-null   float64
 13  Tail_Number               10

In [ ]:
#비행시간 = 도착예정시간 - 출발예정시간 // 문제점) 저녁에 출발하여 다음날 아침에 도착한다고 하면 예) 2158출발 515도착 할 경우 --> 이건 그냥 뒤에 2400을 더해서 ?

In [17]:
train_df['flight_time'] = train_df['Estimated_Arrival_Time']- train_df['Estimated_Departure_Time'] #도착예정시간 - 출발예정시간  = 비행시간 / 도착예정시간이나 출발예정시간 둘 중 하나라도 nan값이 존재하면  비행시간도 결측치로 존재 

In [18]:
train_df[train_df['flight_time'].isna()].head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,flight_time
0,TRAIN_000000,4,15,NaN,NaN,13851,Oklahoma,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN
5,TRAIN_000005,4,13,1545.0,NaN,11618,NaN,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed,NaN
20,TRAIN_000020,7,2,620.0,NaN,10821,Maryland,10721,Massachusetts,369.0,Southwest Airlines Co.,WN,19393.0,N292WN,NaN,NaN
24,TRAIN_000024,11,28,1030.0,NaN,15304,Florida,10599,Alabama,459.0,Southwest Airlines Co.,WN,19393.0,N921WN,NaN,NaN
25,TRAIN_000025,12,24,NaN,2020.0,15023,NaN,14747,Washington,618.0,Horizon Air,AS,19687.0,N451QX,NaN,NaN


In [19]:
train_df['flight_time'].isna().sum() 

206371

In [21]:
flight_df = train_df[['ID','Estimated_Departure_Time','Estimated_Arrival_Time','flight_time','Distance']]

In [22]:
flight_df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,flight_time,Distance
0,TRAIN_000000,NaN,NaN,NaN,419.0
1,TRAIN_000001,740.0,1024.0,284.0,1250.0
2,TRAIN_000002,1610.0,1805.0,195.0,544.0
3,TRAIN_000003,905.0,1735.0,830.0,2454.0
4,TRAIN_000004,900.0,1019.0,119.0,250.0
...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,307.0,678.0
999996,TRAIN_999996,920.0,1028.0,108.0,223.0
999997,TRAIN_999997,800.0,1340.0,540.0,1642.0
999998,TRAIN_999998,1613.0,1824.0,211.0,214.0


In [35]:
flight_df['flight_time'] = flight_df['flight_time'].apply(lambda x : x+2400 if x<0 else x)

<ipython-input-35-b29c8eb8eddd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_df['flight_time'] = flight_df['flight_time'].apply(lambda x : x+2400 if x<0 else x)


In [43]:
i = 2375
a = a = i // 100
b = ( i - ( a * 100 )) //10
b

7

In [47]:
j = 100
d = j // 100

e = j % 100 // 10

f = j % 10
e

0

In [ ]:
#분 시간이 60 이상일 경우 시간으로 변환해 줘야 하는데 십의 자리 숫자만 선택이 가능할까 

In [48]:
flight_df['flight_time'] = flight_df['flight_time'].apply(lambda x : x+40 if x%100//10>=6 else x)

<ipython-input-48-d1b190081c96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_df['flight_time'] = flight_df['flight_time'].apply(lambda x : x+40 if x%100//10>=6 else x)


In [49]:
flight_df #비행시간 계산완료

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,flight_time,Distance
0,TRAIN_000000,NaN,NaN,NaN,419.0
1,TRAIN_000001,740.0,1024.0,324.0,1250.0
2,TRAIN_000002,1610.0,1805.0,235.0,544.0
3,TRAIN_000003,905.0,1735.0,830.0,2454.0
4,TRAIN_000004,900.0,1019.0,119.0,250.0
...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,307.0,678.0
999996,TRAIN_999996,920.0,1028.0,108.0,223.0
999997,TRAIN_999997,800.0,1340.0,540.0,1642.0
999998,TRAIN_999998,1613.0,1824.0,211.0,214.0


1. 만약 비행시간을 잘 넣었다고 치면 --> 거리 평균과 비행시간 평균 구해서 --> 평균비행시간을 거리평균으로 나눠 1km당 비행시간을 구함 --> 비행시간이 나와있지 않던 데이터에 값 채워넣기    

2. 거리에 따른 비행시간 회귀 모델 

In [ ]:
#단순 선형 회귀모델

# 모델 임포트 
from sklearn.linear_model import LinearRegression

# 모델 인스턴스 생성
lin_reg = LinearRegression()

# 모델 피팅
lin_reg.fit(X,y)

# 모델 예측
y_pred = lin_reg.predict(X)
y_pred

# 시각화
plt.scatter(X, y)
plt.plot(X, y_pred, 'r-')

print("모델의 기울기:", end = ' ')
print(lin_reg.coef_[0])

print("모델의 절편:", end = ' ')
lin_reg.intercept_
